In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

Load & split the dataset

In [ ]:
df = pd.read_excel("../7680x321.xlsx")

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# reshape to match lstm input format: (samples, time steps, features)
X = X.reshape(-1, 6, 320) # 6 time steps per sequence; shape: (1280, 6, 320)
y = y[::6] # select every 6th label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

Define the LSTM-CNN architecture

In [ ]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(6, X_train.shape[2])),

    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),

    Flatten(),

    Dense(32, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])

Compile the model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Train the model

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop])

Evaluate the model

In [ ]:
model.evaluate(X_test, y_test)

Save the model

In [ ]:
model.save("../models/lstm_cnn.keras")